# Model Comparison in The Domain of Brain Tumor Image Classification
For our final project for Spring 2025, CS 4644 - Deep Learning, we analyze and compare the results from three unique models:
1. 3D CNN - Turning 2D images into 3D datapoints to reconstruct a full brain image.
2. ResNet18 - Applying transfer learning by taking a pretrained ResNet-18 model (trained on ImageNet) and adapting it to MRI scans through the fine-tuning of a final fully connected layer.
3. Inception - Applying transfer learning in the same way as ResNet, but for another popular and successful model.

The following code allows the reader to experiment with these 3 models and observe their results.

# Step 0: Get necessary imports, set global variables, and setup dataset

In [ ]:
# Basic imports
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import torch
from PIL import Image
import matplotlib.pyplot as plt

# Model imports
from src.models import (
    CNN_3D,
    MyResNet,
    MyInception
)
# Helper imports
from utils.utils import save_trained_model_weights
from utils.dataset_utils import prepare_dataset
from src import (
    Trainer,
    get_optimizer
)
from data.data_transforms import (
    get_fundamental_transforms,
    get_fundamental_normalization_transforms,
    get_fundamental_augmentation_transforms,
    get_all_transforms,
)
from utils.confusion_matrix import (
    generate_confusion_data,
    generate_confusion_matrix,
    plot_confusion_matrix,
    get_pred_images_for_target,
    generate_and_plot_confusion_matrix,
    generate_and_plot_accuracy_table,
)

# Global variables
raw_data_path = "../data/raw/"
data_path = "../data/processed/"
model_path = "../src/models"

batch_size = 32
num_classes = 3

%load_ext autoreload
%autoreload 2

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


### Donwload the dataset, preprocess it, and compute statistics

In [6]:
dataset_mean, dataset_std = prepare_dataset(
    raw_data_path=raw_data_path,
    pickle_path= os.path.join(raw_data_path, "brain_tumor_mri/new_dataset/training_data.pickle"),
    processed_data_path=data_path,
)

FileNotFoundError: Kaggle API token not found. Please follow these steps:
1. Visit https://www.kaggle.com/account
2. Click 'Create New API Token'
3. type the following into your terminal if on MacOS/Linux:
mkdir -p ~/.kaggle
mv /path/to/downloaded/kaggle.json ~/.kaggle/
chmod 600 ~/.kaggle/kaggle.json
4. type the following into your terminal if on Windows:
mkdir %USERPROFILE%\.kaggle
move %USERPROFILE%\Downloads\kaggle.json %USERPROFILE%\.kaggle\ 


In [4]:
model_resnet = MyResNet(num_classes=num_classes)
print(model_resnet)

MyResNet(
  (conv_layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

# Step 1: Test 3D Convolutional Nerual Network

In [ ]:
inp_size = (512,512) # TODO: Double check this

In [ ]:
model_cnn = CNN_3D(num_classes=num_classes)

cnn_optimizer_config = {"optimizer_type": "adam", "lr": 1e-3, "weight_decay": 1e-8} # Tune these
cnn_optimizer = get_optimizer(model_cnn, cnn_optimizer_config)

cnn_trainer = Trainer(
    data_dir=data_path,
    model=model_cnn,
    optimizer=cnn_optimizer,
    model_dir=os.path.join(model_path, "CNN_3D"),
    train_data_transforms=get_all_transforms(inp_size, [dataset_mean], [dataset_std]),
    val_data_transforms=get_fundamental_normalization_transforms(
        inp_size, [dataset_mean], [dataset_std]
    ),
    batch_size=batch_size,
    load_from_disk=False,
    cuda=torch.cuda.is_available(),
)

In [ ]:
%%time
cnn_trainer.run_training_loop(num_epochs=5)

In [ ]:
cnn_trainer.plot_loss_history()
cnn_trainer.plot_accuracy()

cnn_train_accuracy = cnn_trainer.train_accuracy_history[-1]
cnn_validation_accuracy = cnn_trainer.validation_accuracy_history[-1]
print(
    "Train Accuracy = {}; Validation Accuracy = {}".format(
        cnn_train_accuracy, cnn_validation_accuracy
    )
)

In [ ]:
save_trained_model_weights(model_cnn, out_dir=os.path.join(model_path, "CNN_3D"))

# Step 2: Test ResNet Pretrained Model

In [ ]:
inp_size = (224,224) # Double check

In [ ]:
model_resnet = MyResNet(num_classes=num_classes)

resnet_optimizer_config = {"optimizer_type": "adam", "lr": 1e-3, "weight_decay": 1e-8} # Tune these
resnet_optimizer = get_optimizer(model_resnet, resnet_optimizer_config)

resnet_trainer = Trainer(
    data_dir=data_path,
    model=model_resnet,
    optimizer=resnet_optimizer,
    model_dir=os.path.join(model_path, "ResNet"),
    train_data_transforms=get_all_transforms(inp_size, [dataset_mean], [dataset_std]),
    val_data_transforms=get_fundamental_normalization_transforms(
        inp_size, [dataset_mean], [dataset_std]
    ),
    batch_size=batch_size,
    load_from_disk=False,
    cuda=torch.cuda.is_available(),
)

In [ ]:
%%time
resnet_trainer.run_training_loop(num_epochs=5)

In [ ]:
save_trained_model_weights(model_resnet, out_dir=os.path.join(model_path, "ResNet"))

# Step 3: Test Inception Pretrained Model

In [ ]:
inp_size = (299,299) # Double check

In [ ]:
model_inception = MyInception(num_classes=num_classes)

inception_optimizer_config = {"optimizer_type": "adam", "lr": 1e-3, "weight_decay": 1e-8} # Tune these
inception_optimizer = get_optimizer(model_inception, inception_optimizer_config)

inception_trainer = Trainer(
    data_dir=data_path,
    model=model_inception,
    optimizer=inception_optimizer,
    model_dir=os.path.join(model_path, "Inception"),
    train_data_transforms=get_all_transforms(inp_size, [dataset_mean], [dataset_std]),
    val_data_transforms=get_fundamental_normalization_transforms(
        inp_size, [dataset_mean], [dataset_std]
    ),
    batch_size=batch_size,
    load_from_disk=False,
    cuda=torch.cuda.is_available(),
)

In [ ]:
%%time
inception_trainer.run_training_loop(num_epochs=5)

In [ ]:
save_trained_model_weights(model_inception, out_dir=os.path.join(model_path, "Inception"))

# Step 4: Analyze Graphs and Final Accuracies

### Loss & Accuracy Graphs

In [ ]:
cnn_trainer.plot_loss_history()
cnn_trainer.plot_accuracy()

cnn_train_accuracy = cnn_trainer.train_accuracy_history[-1]
cnn_validation_accuracy = cnn_trainer.validation_accuracy_history[-1]
print(
    "Train Accuracy = {}; Validation Accuracy = {}".format(
        cnn_train_accuracy, cnn_validation_accuracy
    )
)

In [ ]:
resnet_trainer.plot_loss_history()
resnet_trainer.plot_accuracy()

resnet_train_accuracy = resnet_trainer.train_accuracy_history[-1]
resnet_validation_accuracy = resnet_trainer.validation_accuracy_history[-1]
print(
    "Train Accuracy = {}; Validation Accuracy = {}".format(
        resnet_train_accuracy, resnet_validation_accuracy
    )
)

In [ ]:
inception_trainer.plot_loss_history()
inception_trainer.plot_accuracy()

inception_train_accuracy = inception_trainer.train_accuracy_history[-1]
inception_validation_accuracy = inception_trainer.validation_accuracy_history[-1]
print(
    "Train Accuracy = {}; Validation Accuracy = {}".format(
        inception_train_accuracy, inception_validation_accuracy
    )
)

### Confusion Matrices

In [ ]:
generate_and_plot_confusion_matrix(model_cnn, cnn_trainer.val_dataset, use_cuda=torch.cuda.is_available())

In [ ]:
generate_and_plot_confusion_matrix(model_resnet, resnet_trainer.val_dataset, use_cuda=torch.cuda.is_available())

In [ ]:
generate_and_plot_confusion_matrix(model_inception, inception_trainer.val_dataset, use_cuda=torch.cuda.is_available())

### Analyze errors that occurred from confusion matrix

In [ ]:
trainer = resnet_trainer # Change this
model = model_resnet # Change this

# Analyze confusion matrix and change these to observe results
predicted_class_num = 0
true_class_num = 0

correct_class = [k for k, v in trainer.val_dataset.class_dict.items() if v == true_class_num][0]
pred_class = key = [k for k, v in trainer.val_dataset.class_dict.items() if v == predicted_class_num][0]
print(trainer.val_dataset.class_dict)

paths = get_pred_images_for_target(model, trainer.val_dataset, predicted_class_num, true_class_num, torch.cuda.is_available())
max_count = 10
count = 0
for path in paths:
    img = Image.open(path).convert(mode='L')
    if (count != max_count):
        plt.imshow(img, cmap='gray')
        plt.title(f'Image of {correct_class}, misclassified as {pred_class}')
        plt.axis('off')  # Removes axis ticks
        plt.show()
        count += 1